In [1]:
import jwt
import msal

In [2]:
# DocHuntVue stuff
client_id = '7a0d396a-c9aa-4a65-8bff-e906fef7b0c7'
client_credential = '24597952-3fe5-42be-ab7b-6308311158dd'
authority = (
    'https://clunacy.b2clogin.com/clunacy.onmicrosoft.com/'
    'B2C_1_dochuntsignupsignin'
)

In [3]:
client = msal.ConfidentialClientApplication(
    client_id,
    client_credential=client_credential,
    authority=authority,
)

In [4]:
response = client.acquire_token_for_client(['results.read.all'])

In [5]:
response

{'error': 'unsupported_grant_type',
 'error_description': 'AADB2C90086: The supplied grant_type [client_credentials] is not supported.\r\nCorrelation ID: d16247da-04f1-436f-9b7e-f31d5f2fe788\r\nTimestamp: 2021-02-02 15:29:03Z\r\n'}

In [6]:
# Fresh start: use a token we pulled ourselves
import getpass
id_token = getpass.getpass('ID Token: ')

In [7]:
token = id_token.strip('"')
if token.startswith('Bearer '):
    token = token[7:]

In [8]:
jwt.__version__

'2.0.1'

In [9]:
header = jwt.get_unverified_header(token)

In [10]:
header

{'typ': 'JWT',
 'alg': 'RS256',
 'kid': 'X5eXk4xyojNFum1kl2Ytv8dlNP4-c57dO6QGTVBwaNk'}

In [11]:
import urllib.request
contents = urllib.request.urlopen(f'{authority}/discovery/keys').read()

In [12]:
import json
disc_keys = json.loads(contents.decode())
disc_keys

{'keys': [{'kid': 'X5eXk4xyojNFum1kl2Ytv8dlNP4-c57dO6QGTVBwaNk',
   'nbf': 1493763266,
   'use': 'sig',
   'kty': 'RSA',
   'e': 'AQAB',
   'n': 'tVKUtcx_n9rt5afY_2WFNvU6PlFMggCatsZ3l4RjKxH0jgdLq6CScb0P3ZGXYbPzXvmmLiWZizpb-h0qup5jznOvOr-Dhw9908584BSgC83YacjWNqEK3urxhyE2jWjwRm2N95WGgb5mzE5XmZIvkvyXnn7X8dvgFPF5QwIngGsDG8LyHuJWlaDhr_EPLMW4wHvH0zZCuRMARIJmmqiMy3VD4ftq4nS5s8vJL0pVSrkuNojtokp84AtkADCDU_BUhrc2sIgfnvZ03koCQRoZmWiHu86SuJZYkDFstVTVSR0hiXudFlfQ2rOhPlpObmku68lXw-7V-P7jwrQRFfQVXw'}]}

In [13]:
jwk = [key for key in disc_keys['keys'] if key['kid'] == header['kid']].pop()
jwk

{'kid': 'X5eXk4xyojNFum1kl2Ytv8dlNP4-c57dO6QGTVBwaNk',
 'nbf': 1493763266,
 'use': 'sig',
 'kty': 'RSA',
 'e': 'AQAB',
 'n': 'tVKUtcx_n9rt5afY_2WFNvU6PlFMggCatsZ3l4RjKxH0jgdLq6CScb0P3ZGXYbPzXvmmLiWZizpb-h0qup5jznOvOr-Dhw9908584BSgC83YacjWNqEK3urxhyE2jWjwRm2N95WGgb5mzE5XmZIvkvyXnn7X8dvgFPF5QwIngGsDG8LyHuJWlaDhr_EPLMW4wHvH0zZCuRMARIJmmqiMy3VD4ftq4nS5s8vJL0pVSrkuNojtokp84AtkADCDU_BUhrc2sIgfnvZ03koCQRoZmWiHu86SuJZYkDFstVTVSR0hiXudFlfQ2rOhPlpObmku68lXw-7V-P7jwrQRFfQVXw'}

In [14]:
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization


def ensure_bytes(key):
    if isinstance(key, str):
        key = key.encode('utf-8')
    return key


def decode_value(val):
    decoded = base64.urlsafe_b64decode(ensure_bytes(val) + b'==')
    return int.from_bytes(decoded, 'big')


def rsa_pem_from_jwk(jwk):
    return RSAPublicNumbers(
        n=decode_value(jwk['n']),
        e=decode_value(jwk['e'])
    ).public_key(default_backend()).public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

In [15]:
public_key = rsa_pem_from_jwk(jwk)

In [16]:
decoded = jwt.decode(
    token,
    public_key,
    verify=True,
    algorithms=['RS256'],
    audience=[client_id],
)

In [17]:
decoded

{'exp': 1612283158,
 'nbf': 1612279558,
 'ver': '1.0',
 'iss': 'https://clunacy.b2clogin.com/07f0848e-60d0-4c23-8482-cebd7ab04d72/v2.0/',
 'sub': '96e95f36-5bfd-4f8a-901b-0df83183bd05',
 'aud': '7a0d396a-c9aa-4a65-8bff-e906fef7b0c7',
 'nonce': '5982f4b6-70e1-4210-bb6f-ef5b2854b2ea',
 'iat': 1612279558,
 'auth_time': 1612279545,
 'country': 'United States',
 'given_name': 'Brian',
 'family_name': 'McClune',
 'name': 'Brian McClune',
 'tfp': 'B2C_1_dochuntsignupsignin'}